<a href="https://colab.research.google.com/github/GusevMihail/seamless_textute_generator/blob/master/batch_image_resizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import albumentations as albu

In [2]:
import os.path
import sys
if 'google' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    !mkdir data
    !mkdir data/train
    if os.path.exists('data/train'):
        !cp /content/drive/MyDrive/Colab/seamless_textute_generator/data/concrete_maps.zip data
        !unzip -q -n data/concrete_maps.zip -d data/train

Mounted at /content/drive


In [3]:
data_folder = r'data/train'
image_names = tuple(f for f in os.listdir(data_folder) if os.path.isfile(os.path.join(data_folder, f)))

In [ ]:
!mkdir data/resize
out_folder = r'data/resize'

In [ ]:
%%time

for f in os.listdir(data_folder):
    f = os.path.join(data_folder, f)
    if os.path.isfile(f):
        image = cv2.imread(f)
        print(image.shape)
        r = cv2.resize(image, (1000,568))
        print(r.shape)

    break


(5000, 8800, 3)
(568, 1000, 3)
(5000, 8800, 3)
(568, 1000, 3)
(5000, 8800, 3)
(568, 1000, 3)
(5000, 8800, 3)
(568, 1000, 3)
(5000, 8800, 3)
(568, 1000, 3)
(5000, 8800, 3)
(568, 1000, 3)
1 loop, best of 5: 659 ms per loop


In [ ]:
%%time

for f in os.listdir(data_folder):
    f = os.path.join(data_folder, f)
    if os.path.isfile(f):
        print('READING')
        %time image = cv2.imread(f)
        print(image.shape)
        transform = albu.LongestMaxSize(max_size=1000, interpolation=cv2.INTER_CUBIC)
        print('RESIZE')
        %time r = transform(image=image)['image']
        print(r.shape)

    break

READING
CPU times: user 681 ms, sys: 9.64 ms, total: 690 ms
Wall time: 700 ms
(5000, 8800, 3)
RESIZE
CPU times: user 44 ms, sys: 961 µs, total: 45 ms
Wall time: 22.7 ms
(568, 1000, 3)
CPU times: user 728 ms, sys: 10.8 ms, total: 739 ms
Wall time: 727 ms


In [ ]:
%%time
MAX_SIZE = 1000

for f in os.listdir(data_folder):
    in_f = os.path.join(data_folder, f)
    if os.path.isfile(in_f):
        image = cv2.imread(in_f)
        transform = albu.LongestMaxSize(max_size=MAX_SIZE, interpolation=cv2.INTER_CUBIC)
        resized = transform(image=image)['image']
        out_f = os.path.join(out_folder, f)
        cv2.imwrite(out_f, resized)
    

CPU times: user 26.5 s, sys: 418 ms, total: 26.9 s
Wall time: 25.8 s


In [ ]:
for f in os.listdir(out_folder):
    f = os.path.join(out_folder, f)
    if os.path.isfile(f):
        print('READING RESIZED IMAGE')
        %time image = cv2.imread(f)
        break

READING RESIZED IMAGE
CPU times: user 14.2 ms, sys: 6 µs, total: 14.2 ms
Wall time: 15.7 ms


In [ ]:
!zip -j -r -q data/concrete_maps_1K.zip data/resize/* 

In [ ]:
!cp data/concrete_maps_1K.zip /content/drive/MyDrive/Colab/seamless_textute_generator/data

Далее следуют тесты


In [ ]:
!mkdir data/test
!unzip -q data/concrete_maps_1K.zip -d data/test

In [ ]:
!rm -r data/test

In [ ]:
!rm data/concrete_maps_1K.zip

In [ ]:
!man zip

In [5]:
import numpy as np

In [9]:
w = 5050

In [37]:
w = 3900
w = w-1000
ixs = np.linspace(0, w, num=w//1000+2, endpoint=True, dtype=np.int16)
for i in ixs:
    print(i, i+999)

0 999
950 1949
1900 2899


In [44]:
# from math import round
w = 5050
c = round(w / 1000)
c

5

In [ ]:
%%time
OUT_W = 1000
OUT_H = 1000

def split_image_side(in_side, out_side):
    return np.linspace(start=0,
                       stop=in_side - out_side,
                       num=in_space / out_space + 1,
                       dtype=np.int16)

for f in os.listdir(data_folder):
    in_f = os.path.join(data_folder, f)
    if os.path.isfile(in_f):
        image = cv2.imread(in_f)
        hight, width, channels = image.shape
        i = 0
        for h in split_image_side(hight, OUT_H):
            for w in split_image_side(width, OUT_W):
                i += 1
                resized = image[h:h + OUT_H, w:w + OUT_W]
                #TODO написать разбивку имени файла на имя и расширение,
                # чтобы добавлять суфикс к имени перед расширением.
                out_f = os.path.join(out_folder, f, )
                cv2.imwrite(out_f, resized)